In [16]:
import random
import numpy as np
import sys
from random import randint
from data_load import *
def getLabelLinks(fn):
    return processLines(fn,lambda x,y:addFeature(x,y.rstrip().split('\t')),list())
in_dir= "/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN"
a=MentionData('/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/train_x_new.txt',
              "/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/train_y.txt",
             in_dir+"/feature.txt",in_dir+"/type.txt")
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [12]:
%%cython
import numpy as np

cimport numpy as np
from random import randint
import sys
import cython

import math
from libc.stdlib cimport malloc, free

from libc.math cimport exp
from libc.math cimport log
from libc.math cimport sqrt
from libc.string cimport memset
import random
# scipy <= 0.15

import scipy.linalg.blas as fblas
ctypedef np.float32_t REAL_t
cdef int ONE = 1


REAL = np.float32
cdef extern from "/Users/mayk/working/figer/baseline/PLE/Model/warp/voidptr.h":
    void* PyCObject_AsVoidPtr(object obj)
DEF MAX_SENTENCE_LEN = 10000
ctypedef void (*scopy_ptr) (const int *N, const float *X, const int *incX, float *Y, const int *incY) nogil
ctypedef void (*saxpy_ptr) (const int *N, const float *alpha, const float *X, const int *incX, float *Y, const int *incY) nogil
ctypedef float (*sdot_ptr) (const int *N, const float *X, const int *incX, const float *Y, const int *incY) nogil
ctypedef double (*dsdot_ptr) (const int *N, const float *X, const int *incX, const float *Y, const int *incY) nogil
ctypedef double (*snrm2_ptr) (const int *N, const float *X, const int *incX) nogil
ctypedef void (*sscal_ptr) (const int *N, const float *alpha, const float *X, const int *incX) nogil


cdef scopy_ptr scopy = <scopy_ptr>PyCObject_AsVoidPtr(fblas.scopy._cpointer)  # y = x
cdef saxpy_ptr saxpy=<saxpy_ptr>PyCObject_AsVoidPtr(fblas.saxpy._cpointer)  # y += alpha * x
cdef sdot_ptr sdot=<sdot_ptr>PyCObject_AsVoidPtr(fblas.sdot._cpointer)  # float = dot(x, y)
cdef dsdot_ptr dsdot=<dsdot_ptr>PyCObject_AsVoidPtr(fblas.sdot._cpointer)  # double = dot(x, y)
cdef snrm2_ptr snrm2=<snrm2_ptr>PyCObject_AsVoidPtr(fblas.snrm2._cpointer)  # sqrt(x^2)
cdef sscal_ptr sscal=<sscal_ptr>PyCObject_AsVoidPtr(fblas.sscal._cpointer) # x = alpha * x
DEF EXP_TABLE_SIZE = 10000
DEF MAX_EXP = 50

cdef REAL_t[EXP_TABLE_SIZE] EXP_TABLE
cdef REAL_t[EXP_TABLE_SIZE] LOG_TABLE

cdef REAL_t ONEF = <REAL_t>1.0

# for when fblas.sdot returns a double
cdef REAL_t our_dot_double(const int *N, const float *X, const int *incX, const float *Y, const int *incY) nogil:
    return <REAL_t>dsdot(N, X, incX, Y, incY)

# for when fblas.sdot returns a float
cdef REAL_t our_dot_float(const int *N, const float *X, const int *incX, const float *Y, const int *incY) nogil:
    return <REAL_t>sdot(N, X, incX, Y, incY)

# for when no blas available
cdef REAL_t our_dot_noblas(const int *N, const float *X, const int *incX, const float *Y, const int *incY) nogil:
    # not a true full dot()-implementation: just enough for our cases
    cdef int i
    cdef REAL_t a
    a = <REAL_t>0.0
    for i from 0 <= i < 50 by 1:
        a += X[i] * Y[i]
    return a

# for when no blas available
cdef void our_saxpy_noblas(const int *N, const float *alpha, const float *X, const int *incX, float *Y, const int *incY) nogil:
    cdef int i
    for i from 0 <= i < N[0] by 1:
        Y[i * (incY[0])] = (alpha[0]) * X[i * (incX[0])] + Y[i * (incY[0])]
cdef REAL_t cvdot(vec1,vec2,size):
    cdef int csize = size
    f= dsdot(&csize,<REAL_t *>(np.PyArray_DATA(vec1)),&ONE,<REAL_t *>(np.PyArray_DATA(vec2)),&ONE)
    return f
def csaxpy(vec1,vec2,alpha,size):
    cdef int csize = size
    cdef float calpha = alpha
    f= our_saxpy_noblas(&csize,&calpha,<REAL_t *>(np.PyArray_DATA(vec1)),&ONE,<REAL_t *>(np.PyArray_DATA(vec2)),&ONE)
    return f
cdef REAL_t crank(int k):
    cdef REAL_t loss = 0.
    cdef int i = 1
    for i in xrange(1,k+1):
        loss += ONEF/i
    return loss
cdef REAL_t vsum(REAL_t *vec,int *size):
    cdef int i
    cdef REAL_t product
    product = <REAL_t>0.0
    for i from 0 <= i < size[0] by 1:
        product += vec[i] * vec[i]
    return np.sqrt(product)
def cnorm(vec):
    cdef int size
    size  = len(vec)
    return vsum(<REAL_t *>(np.PyArray_DATA(vec)),&size)
def init():
    for i in range(EXP_TABLE_SIZE):
        EXP_TABLE[i] = <REAL_t>exp((i / <REAL_t>EXP_TABLE_SIZE * 2 - 1) * MAX_EXP)
        EXP_TABLE[i] = <REAL_t>(EXP_TABLE[i] / (EXP_TABLE[i] + 1))

cdef void divide(REAL_t *vec, const float *alpha, const int *size):
    cdef int i
    cdef REAL_t product
    for i from 0 <= i < size[0] by 1:
        vec[i] = vec[i]/alpha[0]
def cdivide(vec,alpha):
    cdef int size
    size  = len(vec)
    cdef float r = alpha
    divide(<REAL_t *>(np.PyArray_DATA(vec)),&r,&size)
init()


In [13]:
tuples = getLabelLinks('/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/type_type_kb.txt')
tuples = [ (int(t[0]),int(t[1]),float(t[2]))for t in tuples]

In [9]:
def softmax_labelCorr_gradient(B,tup,weight,size,lr=0.01):
    #print B
    #print B[0]-B[9]

    dA = np.zeros(size,dtype=REAL)
    random.seed(1)
    x = np.mean(A[inst.features],axis=0)
    cdef REAL_t error = 0
    cdef REAL_t s1,s2,logs1,logs2,g
    cdef clr = lr
    cdef int neg_num = len(inst.negative_labels)
    cdef REAL_t norm
    cdef REAL_t pos = ONEF
    cdef int csize =size
    for l in inst.sparse_labels:
        logs1= cvdot(x,B[l],50)
        if  logs1 <= -MAX_EXP or  logs1 >= MAX_EXP:
            continue
        s1 = EXP_TABLE[<int>((logs1 + MAX_EXP) * (EXP_TABLE_SIZE / MAX_EXP / 2))]
        error -=logs1
        g = (pos - s1) * clr
        csaxpy(B[l],dA,g,50)
        saxpy(&csize, &g, <REAL_t *>(np.PyArray_DATA(x)), &ONE, <REAL_t *>(np.PyArray_DATA(B[l])), &ONE)
        for k in xrange(neg_num):
            nl = inst.negative_labels[randint(0,neg_num-1)]
            logs2 = cvdot(x,B[nl],50)
            if  logs2 <= -MAX_EXP or  logs2 >= MAX_EXP:
                continue
            s2 = EXP_TABLE[<int>((logs2 + MAX_EXP) * (EXP_TABLE_SIZE / MAX_EXP / 2))]
            if s2 > s1:
                g =  (-s2) * clr
                csaxpy(B[nl],dA,g,50)
    
                saxpy(&csize, &g, <REAL_t *>(np.PyArray_DATA(x)), &ONE, <REAL_t *>(np.PyArray_DATA(B[nl])), &ONE)

                error += logs2
    

    for f in inst.features:
        csaxpy(dA,A[f],clr,50)
        norm = cnorm(A[f])
        #if norm >1:
         #   cdivide(A[f],norm)

    for i in xrange(len(B)):
        norm =  cnorm(B[i])
       # if norm >1:
        #    cdivide(B[i],norm)
            #B[i] /=norm
    return error

def label_train(B,insts,gradient,max_it=10):
    cdef float error
    
    for it in xrange(1,max_it+1):
        error = 0.
        for i,inst in enumerate(insts):
            error+=gradient(B,inst,size,lr=lr)
        
            if i % (len(insts)/100+1) ==0:
                sys.stdout.write("\rIteration %d " % (it)+ "trained {0:.0f}%".format(float(i)*100/len(insts))+" Loss:{0:.2f}".format(error))
                sys.stdout.flush()
        sys.stdout.write("\n")

SyntaxError: invalid syntax (<ipython-input-9-7e7a87d0c424>, line 8)

In [25]:
def corrReg(B,tup,weight,size):
    diff = B[tup[0]] - B[tup[1]]
    B[tup[0]] -= weight*diff
    B[tup[1]] += weight*diff
    norm = np.linalg.norm(diff)
    return norm * norm * weight
    

In [23]:
B= np.random.rand(len(a.label2id),50).astype(np.float32)
corrReg(B,tuples[0],0.1,50)

2.8446403